<a href="https://colab.research.google.com/github/JakeAMystery/FORE_ClassWork/blob/main/CPAI/CPAI_Project_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"


In [2]:

# Mount drive at /gdrive:

from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [3]:
import pandas as pd

# 1.1 Call libraries for image processing
#     Another preprocessing option is text and sequence
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# 1.2, Libraries for building sequential CNN model
#      A model is composed of sequence of layered objects
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D
from tensorflow.keras.layers import Activation, Dropout, Flatten, Dense, Input

# 1.5 OS related
import os

# 1.6 For ROC plotting
import matplotlib.pyplot as plt

# 1.7
import numpy as np
# conda install scikit-learn
from sklearn import metrics
import time


In [24]:


# 2. Our constants
# 2.1 Dimensions to which our images will be adjusted
img_width, img_height = 150, 150

# 2.2 Data folder containing all training images, maybe in folders: cats and dogs

train_data_dir = "/gdrive/MyDrive/Capstone Project in AI/data/train"
test_data_dir =  "/gdrive/MyDrive/Capstone Project in AI/data/validation"

# 2.3 What is the total number of training images
#      that should be generated (not what are available)
nb_train_samples = 989

# 2.5 What is the total no of validation samples that should
#     be generated?
nb_validation_samples = 140

# Some hyperparameters

# 2.6 Batch size to train at one go:
batch_size = 32

# 2.7 How many epochs of training?
epochs = 5

# 2.8 No of test samples
test_generator_samples = 300

# 2.9 For test data, what should be batch size
test_batch_size = 32

# 2.10
input_shape = (img_width, img_height, 3)


Model

In [25]:
model_1 = Sequential()
model_1.add(Input(shape=input_shape))

model_1.add(Conv2D(
    filters=64,
    kernel_size= (3,3),
    padding='same',
    activation= 'relu'
))
model_1.add(MaxPool2D())

model_1.add(Conv2D(
    filters=64,
    kernel_size= (3,3),
    activation= 'relu'
))
model_1.add(MaxPool2D())

model_1.add(Conv2D(
    filters=32,
    kernel_size= (3,3),
    activation= 'relu'
))

model_1.add(Flatten())

model_1.summary()


Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_9 (Conv2D)           (None, 150, 150, 64)      1792      
                                                                 
 max_pooling2d_6 (MaxPoolin  (None, 75, 75, 64)        0         
 g2D)                                                            
                                                                 
 conv2d_10 (Conv2D)          (None, 73, 73, 64)        36928     
                                                                 
 max_pooling2d_7 (MaxPoolin  (None, 36, 36, 64)        0         
 g2D)                                                            
                                                                 
 conv2d_11 (Conv2D)          (None, 34, 34, 32)        18464     
                                                                 
 flatten_3 (Flatten)         (None, 36992)            

In [26]:
model_1.add(Dense(4096, activation='relu'))
model_1.add(Dropout(rate=0.5))

model_1.add(Dense(512, activation='relu'))
model_1.add(Dropout(rate=0.5))

model_1.add(Dense(64, activation='relu'))
model_1.add(Dropout(rate=0.5))

model_1.add(Dense(10, activation='softmax'))

model_1.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_9 (Conv2D)           (None, 150, 150, 64)      1792      
                                                                 
 max_pooling2d_6 (MaxPoolin  (None, 75, 75, 64)        0         
 g2D)                                                            
                                                                 
 conv2d_10 (Conv2D)          (None, 73, 73, 64)        36928     
                                                                 
 max_pooling2d_7 (MaxPoolin  (None, 36, 36, 64)        0         
 g2D)                                                            
                                                                 
 conv2d_11 (Conv2D)          (None, 34, 34, 32)        18464     
                                                                 
 flatten_3 (Flatten)         (None, 36992)            

In [27]:

model_1.compile(
    loss='categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)


In [28]:

tr_dtgen = ImageDataGenerator(
                              rescale=1. / 255,      # Normalize colour intensities in 0-1 range
                              shear_range=0.2,       # Shear varies from 0-0.2
                              zoom_range=0.2,
                              horizontal_flip=True,
                              )

In [29]:

train_generator = tr_dtgen.flow_from_directory(
                                               train_data_dir,
                                               target_size=(img_width, img_height),
                                               batch_size=batch_size,
                                               class_mode='categorical')

Found 989 images belonging to 10 classes.


In [30]:
val_dtgen= ImageDataGenerator(rescale=1. / 255)


In [31]:

validation_generator = val_dtgen.flow_from_directory(
                                                     test_data_dir,
                                                     target_size=(img_width, img_height),
                                                     batch_size=batch_size,
                                                     class_mode='categorical')

Found 140 images belonging to 10 classes.


In [32]:
start = time.time()
history = model_1.fit_generator(
                              generator = train_generator,
                              steps_per_epoch=nb_train_samples // batch_size,
                              epochs=epochs,
                              validation_data=validation_generator,
                              verbose = 1,
                              validation_steps=nb_validation_samples // batch_size
                              )

end = time.time()
(end - start)/60

<ipython-input-32-e20d4a23eed5>:2: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = model_1.fit_generator(


Epoch 1/5
30/30 [==============================] - 165s 5s/step - loss: 2.5860 - accuracy: 0.1139 - val_loss: 2.2980 - val_accuracy: 0.1328
Epoch 2/5
30/30 [==============================] - 157s 5s/step - loss: 2.2927 - accuracy: 0.1285 - val_loss: 2.2979 - val_accuracy: 0.1016
Epoch 3/5
30/30 [==============================] - 156s 5s/step - loss: 2.2937 - accuracy: 0.1421 - val_loss: 2.2882 - val_accuracy: 0.1328
Epoch 4/5
30/30 [==============================] - 154s 5s/step - loss: 2.2833 - accuracy: 0.1254 - val_loss: 2.2807 - val_accuracy: 0.1641
Epoch 5/5
30/30 [==============================] - 158s 5s/step - loss: 2.3000 - accuracy: 0.1463 - val_loss: 2.2888 - val_accuracy: 0.1328


13.96287181377411